## Auslesen der Dateien ##

Achtung: Die Dateien aus dem ersten Verzeichnis (hier D:/hka-aqm-m) müssen extrahiert vorliegen (d.h. nicht als zip-Ordner).

hka-aqm-m

            > ... .dat

            > ... .dat

In [15]:
from Preprocessing_M import DataExtractor, DataPreprocessing

directory = r"C:\Users\maxda\Studium\Semester6\IoT\Gebäude_Daten"
new_directory = r"C:\Users\maxda\Studium\Semester6\IoT\Gebäude_Daten"
de = DataExtractor(first_directory=directory, new_directory=new_directory)

df = de.create_df()

Read data successfully.
Data contains 979166 data points and 18 columns.


## Preprocessing ##

df_preprocessed := Datensatz mit Wetterdaten und Jahreszeiten

df_preprocessed_without_weather := Datensatz ohne Wetterdaten, aber mit Jahreszeiten

df_preprocessed_without_seasons := Datensatz mit Wetterdaten, aber ohne Jahreszeiten

In [41]:
label = "CO2"

dp = DataPreprocessing(label = label)

df_preprocessed = dp.preprocess_df(df, rolling_window = "3d", sample_time = "1d")

Number of outliers detected: -72800
Number of normal samples detected: 897857


In [17]:
df_preprocessed.columns

Index(['date_time', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'BLE', 'tavg',
       'tmin', 'tmax', 'wdir', 'wspd', 'wpgt', 'pres', 'room_number',
       'tmp_diff', 'hum_diff', 'VOC_diff', 'vis_diff', 'IR_diff', 'BLE_diff',
       'tavg_diff', 'tmin_diff', 'tmax_diff', 'wdir_diff', 'wspd_diff',
       'wpgt_diff', 'pres_diff', 'year', 'dayofweek', 'hour', 'season',
       'VOC_CO2_ratio'],
      dtype='object')

In [18]:
df_preprocessed_without_weather = df_preprocessed.drop(columns = ['tavg',
       'tmin', 'tmax', 'wdir', 'wspd', 'wpgt', 'pres',  'tavg_diff', 'tmin_diff', 'tmax_diff', 'wdir_diff', 'wspd_diff',
       'wpgt_diff', 'pres_diff'], axis = 1)

df_preprocessed_without_seasons = df_preprocessed.drop(columns = ["season"], axis = 1)

## Feature Engineering ##

In [8]:
import pickle
fe = pickle.load(open("feature_engineerer.pickle", "rb"))

c:\Users\maxda\miniconda3\envs\IoT\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [19]:
from ML_Preparation.Feature_Engineering import *

n_steps = 7

fe = FeatureEngineering(df_preprocessed,
                        label = label, 
                        categorical_features = ["season", "room_number", "dayofweek"],
                        automated_feature_engineering = False)

X_train, X_val, X_test, y_train, y_val, y_test = fe.feature_engineering(steps_to_forecast = n_steps, skip_scale = True)

In [21]:
fe_no_weather = FeatureEngineering(df_preprocessed_without_weather,
                        label = label, 
                        categorical_features = ["season", "room_number", "dayofweek"],
                        automated_feature_engineering = False)

X_train_nowe, X_val_nowe, X_test_nowe, y_train_nowe, y_val_nowe, y_test_nowe = fe_no_weather.feature_engineering(steps_to_forecast = n_steps, skip_scale = True)

fe_no_seasons = FeatureEngineering(df_preprocessed_without_seasons,
                        label = label, 
                        categorical_features = ["room_number", "dayofweek"],
                        automated_feature_engineering = False)

X_train_nose, X_val_nose, X_test_nose, y_train_nose, y_val_nose, y_test_nose = fe_no_seasons.feature_engineering(steps_to_forecast = n_steps, skip_scale = True)

In [22]:
fe.X_train.shape

(4584, 1, 68)

## Modelltraining ##

In [23]:
from Forecasting_Models import CO2_Forecasting_Model
from matplotlib import pyplot

trainierte Modelle laden

In [24]:
import tensorflow as tf
# from tensorflow.keras.models import load_model

train_models = False
label="CO2"
try:
    model = tf.keras.models.load_model(f"{label}_Forecasting_Model.keras")
    # model_no_weather = tf.keras.models.load_model(f"{label}_Forecasting_Model_no_weather.keras")
    # model_no_seasons = tf.keras.models.load_model(f"{label}_Forecasting_Model_no_seasons.keras")
except:
    print("Models couldn't be loaded. Train new models.")
    train_models = True

In [25]:
if train_models:    
    # Instantiate the model
    model = CO2_Forecasting_Model(n_steps, fe)

    history = model.model.fit(fe.X_train, fe.y_train, epochs=50, batch_size = 7, 
                                validation_data = (fe.X_val, fe.y_val),
                                verbose = 0, 
                                shuffle=False
                                )

    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history["val_loss"], label = "val")
    pyplot.title("Training mit Jahreszeiten und Wetterdaten")
    pyplot.legend()
    pyplot.show()



    # model_no_weather = CO2_Forecasting_Model(n_steps, fe_no_weather)

    # history = model_no_weather.model.fit(fe_no_weather.X_train, fe_no_weather.y_train, epochs=50, batch_size=7, 
    #                                     validation_data = (fe_no_weather.X_val, fe_no_weather.y_val),
    #                                     verbose = 0, 
    #                                     shuffle=False
    #                                     )

    # pyplot.plot(history.history['loss'], label='train')
    # pyplot.plot(history.history["val_loss"], label = "val")
    # pyplot.title("Training mit Jahreszeiten und OHNE Wetterdaten")
    # pyplot.legend()
    # pyplot.show()

    # model_no_seasons = CO2_Forecasting_Model(n_steps, fe_no_seasons)

    # history = model_no_seasons.model.fit(fe_no_seasons.X_train, fe_no_seasons.y_train, epochs=50, batch_size = 7, 
    #                                     validation_data = (fe_no_seasons.X_val, fe_no_seasons.y_val),
    #                                     verbose = 0, 
    #                                     shuffle=False
    #                                 )

    # pyplot.plot(history.history['loss'], label='train')
    # pyplot.plot(history.history["val_loss"], label = "val")
    # pyplot.title("Training OHNE Jahreszeiten und mit Wetterdaten")
    # pyplot.legend()
    # pyplot.show()

In [26]:
fe.X_train.shape

(4584, 1, 68)

In [27]:
if train_models:
    model.model.save(f"{label}_Forecasting_Model.keras")
    model_no_weather.model.save(f"{label}_Forecasting_Model_no_weather.keras")
    model_no_seasons.model.save(f"{label}_Forecasting_Model_no_seasons.keras")

    pred = model.model.predict(fe.X_test)
    pred_no_weather = model_no_weather.model.predict(fe_no_weather.X_test)
    pred_no_seasons = model_no_seasons.model.predict(fe_no_seasons.X_test)
else:
    pred = model.predict(fe.X_test)
    # pred_no_weather = model_no_weather.predict(fe_no_weather.X_test)
    # pred_no_seasons = model_no_seasons.predict(fe_no_seasons.X_test)


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step


## Modellevaluation ##

In [28]:
import plotly.express as px
import seaborn as sns
from Evaluator import Evaluator


ev = Evaluator()

for col in range(0, fe.y_test.shape[1]):
    print(f"t +", col)
    print("Modell mit Wetterdaten und Jahreszeiten (rot)")
    ev.evaluate(y_true = fe.y_test[:, col], y_pred = pred[:, col])
    # print("Modell ohne Wetterdaten und mit Jahreszeiten (grün)")
    # ev.evaluate(y_true = fe_no_weather.y_test[:, col], y_pred = pred_no_weather[:, col])
    # print("Modell mit Wetterdaten und ohne Jahreszeiten (violett)")
    # ev.evaluate(y_true = fe_no_seasons.y_test[:, col], y_pred = pred_no_seasons[:, col])

px.line(y = [fe.y_test[:, col], pred[:, col]], #pred_no_weather[:,col], pred_no_seasons[:, col]], 
            labels = {"wide_variable_0": "y_true",
                    "wide_variable_1": "Modell 1 - Vorhersage (Wetterdaten + Jahreszeiten)", 
                   "wide_variable_2": "Modell 2 - Vorhersage (Jahreszeiten)",
                   "wide_variable_3": "Modell 3 - Vorhersage (Wetterdaten)", },             
            title = f"Prediction for CO2(t + {col})  blue = y_true, red = y_pred").show()


t + 0
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  7036.89
RMSE:  83.89
MAPE:  0.13
R2:  -0.39


t + 1
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  6613.19
RMSE:  81.32
MAPE:  0.13
R2:  -0.3


t + 2
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  6565.24
RMSE:  81.03
MAPE:  0.13
R2:  -0.31


t + 3
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  6675.13
RMSE:  81.7
MAPE:  0.13
R2:  -0.33


t + 4
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  6829.6
RMSE:  82.64
MAPE:  0.13
R2:  -0.36


t + 5
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  6756.17
RMSE:  82.2
MAPE:  0.13
R2:  -0.34


t + 6
Modell mit Wetterdaten und Jahreszeiten (rot)
MSE:  6458.44
RMSE:  80.36
MAPE:  0.13
R2:  -0.28




## Deployment bzw. Inferenz mit dem trainierten Modell ##

In [29]:
df = pd.read_parquet("Rohdaten_IoT_mit_date_time_und_snr_konvertiert.parquet")

In [30]:
from ML_Deployment import *

fe.df = df.copy()

deployer = Predictor(data = df, feature_engineering_class_object = fe, label = "CO2", is_forecast = True, roll = True, steps_to_forecast = 2)


In [47]:
deployer.df[["room_number"]]

,room_number
date_time,
2022-06-01,m001
2022-06-02,m001
2022-06-03,m001
2022-06-04,m001
2022-06-20,m104
...,...
2023-09-26,m306
2023-09-26,m301
2023-09-26,m003


In [45]:
deployer.x.shape

(6403, 1, 68)

In [32]:
if train_models:
    forecasted_pred = deployer.predict(x = deployer.x, model = model.model)
else:
    forecasted_pred = deployer.predict(x = deployer.x, model = model)

forecasted_pred

201/201 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


,prediction_t+0,prediction_t+1,prediction_t+2,prediction_t+3,prediction_t+4,prediction_t+5,prediction_t+6,room_number
date_time,,,,,,,,
2022-06-01,544.611389,543.590576,546.047974,542.804077,541.373169,543.143250,541.924377,m102
2022-06-02,488.861450,482.547394,491.170563,487.144623,485.474670,488.117798,493.789795,m102
2022-06-03,538.532349,537.024353,540.003601,537.092712,535.474304,537.059448,536.933716,m102
2022-06-04,523.866516,521.641113,523.778992,522.481995,520.589294,524.216431,524.368164,m102
2022-06-20,554.125061,553.640869,555.155579,551.568604,550.578735,552.349915,549.748230,m700
...,...,...,...,...,...,...,...,...
2023-09-26,550.243408,549.774109,551.721497,548.137756,546.831177,548.934692,546.442932,m202
2023-09-26,538.192017,536.736816,539.742249,536.814453,535.142456,536.839355,536.591980,m202
2023-09-26,555.144653,554.687500,556.130798,552.507935,551.572266,553.300110,550.641785,m113


In [33]:
len(df_preprocessed.room_number.unique())

27

In [36]:
forecasted_pred.room_number.unique()

array(['m102', 'm700', 'mu04', 'm105', 'm104', 'm202', 'm207', 'm206',
       'm201', 'mu10', 'mu06', 'm101', 'm001', 'm002', 'm003', 'm004',
       'm103', 'm113', 'm209'], dtype=object)

In [40]:
date_max = forecasted_pred.index.max()

forecasted_pred[forecasted_pred.index == date_max]

,prediction_t+0,prediction_t+1,prediction_t+2,prediction_t+3,prediction_t+4,prediction_t+5,prediction_t+6,room_number
date_time,,,,,,,,
2023-09-26,536.249207,534.800537,537.702820,534.938721,533.173279,535.280396,534.838928,m105
2023-09-26,534.164551,532.503174,535.553040,532.501099,531.285461,532.340027,532.868286,m700
2023-09-26,555.240356,554.571045,556.190308,552.631226,551.707947,553.314331,551.049377,m202
2023-09-26,539.242126,537.628052,540.552246,537.674988,536.166138,537.524780,537.642456,m202
2023-09-26,497.922424,494.709625,498.049652,496.909821,495.685150,497.237061,500.583679,m700
2023-09-26,493.098572,489.817230,493.294617,492.136108,491.084259,492.182434,495.994751,m206
2023-09-26,540.932007,539.713013,542.392944,539.359558,537.780518,539.623291,538.720581,m700
2023-09-26,554.340881,553.620850,555.295166,551.784058,550.829895,552.434143,550.271729,m700
2023-09-26,518.716125,516.008545,519.359619,517.229980,515.667358,517.620605,519.475525,m104
